# Codigo propio

In [1]:
import lib.custom_metrics as metrics
import lib.evaluation as ev
import lib.plotting as plot
import lib.models as models
import lib.dataset as dt

import matplotlib.pyplot as plt
import imgaug.augmenters as iaa
import tensorflow as tf
import pandas as pd
import numpy as np
import datetime
import os

from importlib import reload

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [3]:
DR_LEVELS_PER_CLASS = [[0], [1,2,3,4]]

IMAGE_SIZE = (540, 540, 3)

# Specify dataset files
TRAIN_FILE = 'DATASET-VALIDATION-10.csv'
VALIDATION_FILE = 'DATASET-VALIDATION-10.csv'

BATCH_SIZE = 32

In [4]:
def read_image(path, label):
    return tf.image.decode_png(tf.io.read_file(path)), label

def normalize(img, label):
    return tf.cast(img, tf.float32) / 255., label

In [6]:
train_csv = pd.read_csv(TRAIN_FILE)

asignations = [-1 for i in DR_LEVELS_PER_CLASS for j in i] # [[0], [1], [2,3,4]] --> [-1, -1, -1, -1, -1]
for i, class_ in enumerate(DR_LEVELS_PER_CLASS):
    for dr_lvl in class_:
        asignations[dr_lvl] = i
            
dr_lvls = [asignations[i] for i in train_csv['DR_level'].tolist()]

# print(asignations)
# print(dr_lvls[:20])

dataset_train = tf.data.Dataset.from_tensor_slices((train_csv['path'].tolist(), tf.keras.utils.to_categorical(dr_lvls, num_classes=len(DR_LEVELS_PER_CLASS))))
dataset_train = dataset_train.map(read_image)

# dataset_train = dataset_train.map(dt.tf_apply_data_augmentation)

dataset_train = dataset_train.map(normalize)
dataset_train = dataset_train.batch(BATCH_SIZE)

#----------------------------------------------

val_csv = pd.read_csv(VALIDATION_FILE)

asignations = [-1 for i in DR_LEVELS_PER_CLASS for j in i] # [[0], [1], [2,3,4]] --> [-1, -1, -1, -1, -1]
for i, class_ in enumerate(DR_LEVELS_PER_CLASS):
    for dr_lvl in class_:
        asignations[dr_lvl] = i
            
dr_lvls = [asignations[i] for i in val_csv['DR_level'].tolist()]

# print(asignations)
# print(dr_lvls[:20])

dataset_val = tf.data.Dataset.from_tensor_slices((val_csv['path'].tolist(), tf.keras.utils.to_categorical(dr_lvls, num_classes=len(DR_LEVELS_PER_CLASS))))
dataset_val = dataset_val.map(read_image)
dataset_val = dataset_val.map(normalize)
dataset_val = dataset_val.batch(BATCH_SIZE)

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(5,5), strides=2, input_shape=IMAGE_SIZE),
    tf.keras.layers.ReLU(), # 268, 268
    tf.keras.layers.Conv2D(16,(7,7), strides=5),
    tf.keras.layers.ReLU(), # 53, 53
    tf.keras.layers.Conv2D(16,(3,3), strides=2),
    tf.keras.layers.ReLU(), # 26, 26
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(24, activation='relu'),
#     tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(len(DR_LEVELS_PER_CLASS), activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 268, 268, 16)      1216      
_________________________________________________________________
re_lu (ReLU)                 (None, 268, 268, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 53, 53, 16)        12560     
_________________________________________________________________
re_lu_1 (ReLU)               (None, 53, 53, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 16)        2320      
_________________________________________________________________
re_lu_2 (ReLU)               (None, 26, 26, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 10816)             0

In [9]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

cbacks = [tf.keras.callbacks.TensorBoard('logs/MBA_Red_Seq_EYEPACS_iguales', histogram_freq=1, write_graph=False)]

In [10]:
# See Tensorboard
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 2790), started 2 days, 21:53:39 ago. (Use '!kill 2790' to kill it.)

In [11]:
model.fit(dataset_train, epochs=200, validation_data=dataset_val, verbose=1, callbacks=cbacks)

Epoch 1/200
  1/222 [..............................] - ETA: 0s - loss: 0.6768 - accuracy: 0.6875WARNING:tensorflow:From /home/alumno/miguel_herrera/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
222/222 [==============================] - 74s 332ms/step - loss: 0.6799 - accuracy: 0.6727 - val_loss: 0.6799 - val_accuracy: 0.6727
Epoch 2/200
222/222 [==============================] - 77s 347ms/step - loss: 0.6799 - accuracy: 0.6727 - val_loss: 0.6799 - val_accuracy: 0.6727
Epoch 3/200
222/222 [==============================] - 75s 338ms/step - loss: 0.6799 - accuracy: 0.6727 - val_loss: 0.6799 - val_accuracy: 0.6727
Epoch 4/200
222/222 [==============================] - 73s 329ms/step - loss: 0.6799 - accuracy: 0.6727 - val_loss: 0.6799 - val_accuracy: 0.6727
Epoch 5/200
222/222 [================

KeyboardInterrupt: 

In [7]:
import pandas as pd

In [10]:
dataset_orig = pd.read_csv('eyepacs.csv')
d = {row.path: row.DR_level for row in dataset_orig.itertuples()}

In [11]:
dataset_usado = pd.read_csv('DATASET-VALIDATION-10.csv')
for row in dataset_usado.itertuples():
    if d[row.path] != row.DR_level:
        print('DR level de una imagen es distinto al que deberia')
        print('imagen:', row.path, 'deberia tener', d[row.path], 'en lugar de', row.DR_level)


In [12]:
dataset_usado = pd.read_csv('DATASET-TEST-10.csv')
for row in dataset_usado.itertuples():
    if d[row.path] != row.DR_level:
        print('DR level de una imagen es distinto al que deberia')
        print('imagen:', row.path, 'deberia tener', d[row.path], 'en lugar de', row.DR_level)